## Solwer DPLL

Podstawowa różnica między poprzednim solwerem a solwerem DPLL to wykonywanie propagacji jednostkowej (ang. unit propagation) oraz ustawiania wartości zmiennych występujących tylko z jednym znakiem przed zejściami rekurencyjnymi

In [1]:
from dimacs import *

In [2]:
def is_clause_correct(C, V):
    return len([l for l in C if l in V and V[l] == 1]) > 0


def simplify_clause(C, V):
    if is_clause_correct(C, V):
        return None
    return [l for l in C if l not in V or V[l] != -1]


def simplifyCNF(cnf, V):
    newCnf = (cnf[0], [])
    for c in cnf[1]:
        simp = simplify_clause(c, V)
        if simp is None:
            continue
        if len(simp) == 0:
            return None
        newCnf[1].append(simp)

    return newCnf


def get_V_with_v_1(V, v):
    new_V = V.copy()
    new_V[v] = 1
    new_V[-v] = -1
    return new_V

## Unit Propagation

Optymalizacja ta polega na tym, że przeglądamy wszystkie klauzule i jeśli znajdziemy klauzulę składającą się z jednego literału, to wiadomo, że należy mu przypisać wartość 1. Oczywiście przypisanie wartości jednemu takiemu literałowi może spowodować, że powstają kolejne, tak więc operację tę należy wykonywać dopóki (a) formuła nie zostanie spełniona, (b) nie dojdziemy do sprzecznośclu, lub (c) nie ma więcej klauzul jednostkowych.

In [3]:
def unit_propagation(cnf, V):
    newCnf = cnf[1].copy()
    newV = V.copy()

    while True:
        one_clause = next((c for c in newCnf if len(c) == 1), None)
        if one_clause is None:
            return (cnf[0], newCnf), newV
        if one_clause[0] in newV and newV[one_clause[0]] == -1:
            return None, newV

        newV[one_clause[0]] = 1
        newV[-one_clause[0]] = -1
        tmp = simplifyCNF((cnf[0], newCnf), newV)
        if tmp is None:
            return None, newV
        newCnf = tmp[1]


def is_in_every_clause(clauses, l):
    for c in clauses:
        if not l in c:
            return False
    return True


def fix_const(cnf, V):
    newCnf = cnf[1].copy()
    newV = V.copy()
    for l in range(-cnf[0], cnf[0] + 1):
        if l == 0:
            continue
        if is_in_every_clause(newCnf, l):
            newV[l] = 1
            newV[-l] = -1
            tmp = simplifyCNF((cnf[0], newCnf), newV)
            if tmp is None:
                return None, newV
            newCnf = tmp[1]

    return (cnf[0], newCnf), newV

In [4]:
def solve(cnf, V):
    if not cnf[1]:
        return V

    cnf, V = unit_propagation(cnf, V)
    if cnf is None:
        return "UNSAT"
    if not cnf[1]:
        return V

    cnf, V = fix_const(cnf, V)
    if cnf is None:
        return "UNSAT"
    if not cnf[1]:
        return V

    v = abs(cnf[1][0][0])
    V1 = get_V_with_v_1(V, v)
    V2 = get_V_with_v_1(V, -v)
    cnf1 = simplifyCNF(cnf, V1)
    cnf2 = simplifyCNF(cnf, V2)

    if cnf1 is not None:
        sol = solve(cnf1, V1)
        if sol != "UNSAT":
            return sol
    if cnf2 is not None:
        sol = solve(cnf2, V2)
        if sol != "UNSAT":
            return sol

    return "UNSAT"

In [8]:
for test in ['5.no.sat', '5.yes.sat', '10.no.sat', '10.yes.sat', '20.no.sat', '20.yes.sat', '30.no.sat', '30.yes.sat']:
    cnf = loadCNF(f'sat/{test}')
    V = {}

    result = solve(cnf, V)
    print(f"SAT solution for test {test} is {str(result)}\n")

SAT solution for test 5.no.sat is UNSAT

SAT solution for test 5.yes.sat is {14: 1, -14: -1, -7: 1, 7: -1, -11: 1, 11: -1, -8: 1, 8: -1, 4: 1, -4: -1, -1: 1, 1: -1, -10: 1, 10: -1, -5: 1, 5: -1, 13: 1, -13: -1, -2: 1, 2: -1, 6: 1, -6: -1, -9: 1, 9: -1, 12: 1, -12: -1, -3: 1, 3: -1}

SAT solution for test 10.no.sat is UNSAT

SAT solution for test 10.yes.sat is {-12: 1, 12: -1, -22: 1, 22: -1, 29: 1, -29: -1, -1: 1, 1: -1, -3: 1, 3: -1, -14: 1, 14: -1, -23: 1, 23: -1, 8: 1, -8: -1, -27: 1, 27: -1, 24: 1, -24: -1, -21: 1, 21: -1, -26: 1, 26: -1, 30: 1, -30: -1, -16: 1, 16: -1, 28: 1, -28: -1, 5: 1, -5: -1, -17: 1, 17: -1, -20: 1, 20: -1, -18: 1, 18: -1, 4: 1, -4: -1, -6: 1, 6: -1, 25: 1, -25: -1, -9: 1, 9: -1, -7: 1, 7: -1, 15: 1, -15: -1, 2: 1, -2: -1, -19: 1, 19: -1, -13: 1, 13: -1, -11: 1, 11: -1, -10: 1, 10: -1}

SAT solution for test 20.no.sat is UNSAT

SAT solution for test 20.yes.sat is {8: 1, -8: -1, -1: 1, 1: -1, -3: 1, 3: -1, -16: 1, 16: -1, -22: 1, 22: -1, -14: 1, 14: -1, 51: 1